<a href="https://colab.research.google.com/github/asking28/offenseval2020/blob/master/offens_2020_greek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip install focal-loss
!pip install keras-tcn==2.8.3
!pip install keras-multi-head

  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=2893029c8e8d91fb11aac8a7dab4c393dfa2c29eafc02e5008091b172672d21b
  Stored in directory: /root/.cache/pip/wheels/bb/df/3f/81b36f41b66e6a9cd69224c70a737de2bb6b2f7feb3272c25e
  Created wheel for keras-self-attention: filename=keras_self_attention-0.41.0-cp36-none-any.whl size=17288 sha256=8242d5e4c097fe87b5f04127f50ae58de647169f4cc2d0733bd3d8b87a29bf20
  Stored in directory: /root/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-multi-head keras-self-attention


In [4]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from keras_multi_head import MultiHeadAttention
from keras_multi_head import MultiHead
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
from focal_loss import BinaryFocalLoss
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
!pip install keras_metrics
import keras_metrics as km

In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
train_data=pd.read_csv(root_path+'/non_english_data/Greek/offenseval-greek-training-v1.tsv',delimiter='\t',quoting=3)

In [0]:
test_data=pd.read_csv(root_path+'/non_english_data/Greek/testset_taska.tsv',delimiter='\t',quoting=3)

In [0]:
test_data_labels=pd.read_csv(root_path+'/non_english_data/Greek/greek-goldlabels.csv',header=None,quoting=3)

In [13]:
print(train_data.head())

     id                                              tweet subtask_a
0  1172  @USER Οι μουσουλμάνες που τις βιάζουν έτσι κ α...       OFF
1  4078  Η Κάτια προσπαθεί να πείσει οτι δεν είναι ελέφ...       NOT
2   135  Καλά γιατί λες ότι, είσαι νέος αφού γεννήθηκες...       NOT
3  9056          Με Φατσεα ξεκινησαμε...... #Kokkinopotami       NOT
4  5344                  #gntmgr Κάτια πόσο γλυκιά, εμετός       NOT


In [14]:
print(test_data_labels.head())

      0    1
0  2707  NOT
1  2251  NOT
2  9814  NOT
3  8949  NOT
4  6913  NOT


In [15]:
print(train_data.shape)

(8743, 3)


In [16]:
print(test_data.shape)
print(test_data.head())

(1544, 2)
     id                                              tweet
0  2707  @USER Θέλω να των δω από εδώ και εμπρός αν δεν...
1  2251  #survivorgr Α Και 60 φορές και με διαφορετικού...
2  9814  Και μου έλεγε η γυναίκα μου το πρωί πάρε την τ...
3  8949                   κατω τα χερια απο τον #κυρανακης
4  6913  @USER μην μας το παιζεις πονοψυχη,κρυφορατσιστ...


## Tokenization and Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [0]:
df_train,df_dev=train_test_split(train_data,test_size=0.1,shuffle=True,random_state=42,stratify=train_data['subtask_a'])

In [0]:
y_train=(df_train['subtask_a']=="OFF").astype(int)
y_dev=(df_dev['subtask_a']=="OFF").astype(int)

In [0]:
y_test=(test_data_labels[1]=="OFF").astype(int)

In [25]:
print(train_data.shape)

(8743, 3)


In [0]:
max_words =5000
max_len = 40
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['tweet'].astype(str))

In [0]:
sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_dev = tok.texts_to_sequences(df_dev['tweet'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Model

In [0]:
word_index=tok.word_index

In [32]:
!pip install keras-self-attention

In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    x1=Embedding(max_words+1,embed_size)(inp1)
    #x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
    #              trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    #x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    # x1=MultiHead(layer=keras.layers.Bidirectional(keras.layers.CuDNNLSTM(units=100), name='LSTM'),
    # layer_num=5,
    # reg_index=[1, 4],
    # reg_slice=(slice(None, None), slice(32, 48)),
    # reg_factor=0.1,
    # name='Multi-Head-Attention')(x1)
    x1 = Bidirectional(CuDNNLSTM(256, return_sequences=True))(x1)
    # x1 = SeqSelfAttention(kernel_regularizer=keras.regularizers.l2(1e-4),
    #                    bias_regularizer=keras.regularizers.l1(1e-4),
    #                    attention_regularizer_weight=1e-4,
    #                    name='Attention')(x1)   
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=300)

In [221]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

tracking <tf.Variable 'Variable_60:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_61:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_62:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_63:0' shape=() dtype=int32> fn


In [222]:
model_bi.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 40, 300)      1500300     input_16[0][0]                   
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 40, 512)      1142784     embedding_16[0][0]               
__________________________________________________________________________________________________
global_max_pooling1d_16 (Global (None, 512)          0           bidirectional_9[0][0]            
___________________________________________________________________________________________

In [0]:
y=(train_data['subtask_a']=="OFF").astype(int)

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y),y)
class_weights=dict(enumerate(class_weights))
class_weights[1]=4

In [225]:
print(class_weights)

{0: 0.6986575035959726, 1: 4}


In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_greek.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [227]:
model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_dev),epochs=2,batch_size=32,callbacks=[es,cp_check_point])

Train on 7868 samples, validate on 875 samples
Epoch 1/2
7868/7868 [==============================] - 4s 566us/step - loss: 0.5312 - acc: 0.7632 - f1_score: 0.1648 - val_loss: 0.4497 - val_acc: 0.8171 - val_f1_score: 0.5916
Epoch 2/2
7868/7868 [==============================] - 3s 405us/step - loss: 0.3572 - acc: 0.8585 - f1_score: 0.7201 - val_loss: 0.4691 - val_acc: 0.8137 - val_f1_score: 0.5879


In [0]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [229]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
# y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(y_dev, y_preds_dl.round(),digits=4))

              precision    recall  f1-score   support

           0     0.8284    0.9329    0.8775       626
           1     0.7529    0.5141    0.6110       249

    accuracy                         0.8137       875
   macro avg     0.7907    0.7235    0.7443       875
weighted avg     0.8069    0.8137    0.8017       875



In [0]:
sequences_test = tok.texts_to_sequences(test_data['tweet'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len,padding='post',truncating='post')

In [0]:
preds=model_bi.predict([sequences_matrix_test])

In [219]:
print(classification_report(y_test, preds.round(),digits=4))


              precision    recall  f1-score   support

           0     0.9660    0.9616    0.9638      1302
           1     0.7984    0.8182    0.8082       242

    accuracy                         0.9391      1544
   macro avg     0.8822    0.8899    0.8860      1544
weighted avg     0.9398    0.9391    0.9394      1544



In [0]:
a=[]
for i in range(len(preds)):
  if preds[i].round()==1:
    a.append("OFF")
  else:
    a.append("NOT")

In [0]:
df=pd.DataFrame({"ids":test_data['id'],"preds":a},index=None)
df.to_csv(root_path+'/non_english_data/Greek/submission.csv',index=False)